In [12]:
#importing libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel #this is used to keep a threshold for our randomforest feature selection
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV #hyperparameter tuning
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import seaborn as sns
import matplotlib 
import numpy as np
import warnings

from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

from yellowbrick.features import RFECV
import statsmodels.formula.api as smf

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

np.random.seed(42)

In [2]:
#importing dataset
data=pd.read_csv('RealPred3.csv')

<h1> Data Cleaning </h1>

In [3]:
#data cleaning
#checking for null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 29 columns):
Gender                                                                                            353 non-null object
Age                                                                                               353 non-null int64
Income                                                                                            353 non-null object
Bodyweight in kg(kilogram)                                                                        353 non-null int64
Sexuality                                                                                         353 non-null object
Virgin                                                                                            353 non-null object
No_of_friends                                                                                     353 non-null int64
SocialFear                                                              

In [4]:
#dropping unwanted columns
cols = ['How do you deal with stressful situation in life? ', 'What kind of help you would want yourself to have when you are facing a psychological problem?',
       'What is that one thing that you would like to improve about yourself?']
data = data.drop(cols,axis = 1)
data.head()

,Gender,Age,Income,Bodyweight in kg(kilogram),Sexuality,Virgin,No_of_friends,SocialFear,employment,EducationStatus,...,Feeling_lost_in_life,selfloathing,any_previous_trauma,Been_a_victim_of_bullying_and_ragging,Alcohol,Drugs,Family_Issues,Any_chronic_disease,Any_attempt_of_Suicide,Selfharm
0,Female,21,0,64,Other,Yes,3,No,Not Employed(Student),Current Student,...,Yes,Yes,No,No,Yearly,Don't do drugs,Yes,No,Once,No
1,Female,21,0,73,Straight,Yes,15,No,Not Employed(Student),Current Student,...,Yes,No,Yes,No,Don't Drink,Don't do drugs,No,No,Few times in past,No
2,Male,20,0,85,Straight,Yes,10,Yes,Not Employed(Student),Current Student,...,Yes,No,No,Yes,Don't Drink,Don't do drugs,No,No,Few times in past,No
3,Female,25,1 lakh to 2 lakh,54,Straight,Yes,5,No,Full time Employee,Passout,...,No,No,No,No,Don't Drink,Don't do drugs,No,No,Few times in past,No
4,Female,21,0,73,Straight,Yes,15,No,Not Employed(Student),Current Student,...,Yes,No,Yes,No,Don't Drink,Don't do drugs,Yes,No,Never,No


<h1> Feature Encoding </h1>

In [5]:
#feature encoding
#making categorical and bin column different and setting the target as suicidal
cate_col = ['Gender', 'Sexuality', 'employment', 'EducationStatus', 'Income','Relationship_Status','Alcohol',
            'Drugs','Any_attempt_of_Suicide' ]
bin_col = ['SocialFear', 'Virgin', 'Insomnia', 'Onantidepressant','Able_to_communicate_oppsex','appetite','Anger_issues',
           'Feeling_lost_in_life','selfloathing','any_previous_trauma',
           'Been_a_victim_of_bullying_and_ragging','Family_Issues','Any_chronic_disease']
#text_col = ['job_title', 'what_help_from_others', 'improve_yourself_how']

pivot = ['Gender', 'Sexuality', 'Virgin','employment', 'EducationStatus', 'Income','Relationship_Status',
         'Alcohol',
         'Drugs','Any_attempt_of_Suicide',  'Able_to_communicate_oppsex',
         'SocialFear', 'Insomnia', 'Onantidepressant','appetite','Anger_issues','Feeling_lost_in_life','selfloathing','any_previous_trauma',
         'Been_a_victim_of_bullying_and_ragging','Family_Issues','Any_chronic_disease']
Target = ['Selfharm']

In [6]:
#Convert Target Variable to 0/1
data['Selfharm'] = np.where(data['Selfharm'].str.contains("Yes"),1,0)

In [7]:
#Print out correlation between categorical variable and target variable as pivot table. 
#The number is probability of attempting suicide.
#it is a simple mean formula applying

for x in pivot:
        print('Survival Correlation by:', x)
        print(data[[x, Target[0]]].groupby(x, as_index=False).mean())
        print('-'*10, '\n')

Survival Correlation by: Gender
              Gender  Selfharm
0             Female  0.152174
1               Male  0.140845
2  Prefer not to say  0.500000
---------- 

Survival Correlation by: Sexuality
  Sexuality  Selfharm
0  Bisexual  0.333333
1     Other  0.000000
2  Straight  0.144928
---------- 

Survival Correlation by: Virgin
  Virgin  Selfharm
0     No  0.202532
1    Yes  0.131387
---------- 

Survival Correlation by: employment
              employment  Selfharm
0     Full time Employee  0.103448
1  Not Employed(Student)  0.146341
2     Part-time Employee  0.217391
3          Self Employed  0.142857
---------- 

Survival Correlation by: EducationStatus
   EducationStatus  Selfharm
0  Current Student  0.152318
1          Dropout  0.000000
2          Passout  0.127660
3         Repeater  0.000000
---------- 

Survival Correlation by: Income
             Income  Selfharm
0                 0  0.149826
1  1 lakh to 2 lakh  0.062500
2       2 to 5 lakh  0.130435
3      5 lakh abov

In [8]:
#Encode the categorical numerical values
encoding = {
            'Income': {'0': 0,'below 1 lakh': 1, '1 lakh to 2 lakh': 2, '2 to 5 lakh': 3, '5 lakh above': 4},
            'Alcohol':{"Don't Drink": 0,'Regularly':1,'Weekly':2,'Monthly':3,'Yearly':4},
            'Drugs':{"Don't do drugs": 0,'Regularly':1,'Weekly':2,'Monthly':3,'Yearly':4},
            'Any_attempt_of_Suicide':{'Never':0,'Once':1,'Few times in past':2,'Refuse to answer':3},
            'EducationStatus':{'Passout':0,'Repeater':1,'Dropout':2,'Current Student':3},
            'Relationship_Status':{'Single':0,'Married':1,'In-relationship':2},
            'employment':{'Not Employed(Student)':0,'Full time Employee':1,'Self Employed':2,'Part-time Employee':3},
            'Sexuality':{'Straight':0,'Bisexual':1,'Other':2},
            'Gender':{'Male':0,'Female':1,'Prefer not to say':2}
           }
           
           #'bodyweight': {'Underweight': 0, 'Normal weight' : 1,'Overweight' : 2,
                         #'Obese': 3 }
           

data.replace(encoding, inplace=True)
data.head()

,Gender,Age,Income,Bodyweight in kg(kilogram),Sexuality,Virgin,No_of_friends,SocialFear,employment,EducationStatus,...,Feeling_lost_in_life,selfloathing,any_previous_trauma,Been_a_victim_of_bullying_and_ragging,Alcohol,Drugs,Family_Issues,Any_chronic_disease,Any_attempt_of_Suicide,Selfharm
0,1,21,0,64,2,Yes,3,No,0,3,...,Yes,Yes,No,No,4,0,Yes,No,1,0
1,1,21,0,73,0,Yes,15,No,0,3,...,Yes,No,Yes,No,0,0,No,No,2,0
2,0,20,0,85,0,Yes,10,Yes,0,3,...,Yes,No,No,Yes,0,0,No,No,2,0
3,1,25,2,54,0,Yes,5,No,1,0,...,No,No,No,No,0,0,No,No,2,0
4,1,21,0,73,0,Yes,15,No,0,3,...,Yes,No,Yes,No,0,0,Yes,No,0,0


In [9]:
#Encode binary variable
bin_col = ['SocialFear', 'Virgin', 'Insomnia', 'Onantidepressant','Able_to_communicate_oppsex','appetite','Anger_issues',
           'Feeling_lost_in_life','selfloathing','any_previous_trauma',
           'Been_a_victim_of_bullying_and_ragging','Family_Issues','Any_chronic_disease']

for i in bin_col:
  data[i] = np.where(data[i].str.contains("Yes"),1,0)
data.head()

,Gender,Age,Income,Bodyweight in kg(kilogram),Sexuality,Virgin,No_of_friends,SocialFear,employment,EducationStatus,...,Feeling_lost_in_life,selfloathing,any_previous_trauma,Been_a_victim_of_bullying_and_ragging,Alcohol,Drugs,Family_Issues,Any_chronic_disease,Any_attempt_of_Suicide,Selfharm
0,1,21,0,64,2,1,3,0,0,3,...,1,1,0,0,4,0,1,0,1,0
1,1,21,0,73,0,1,15,0,0,3,...,1,0,1,0,0,0,0,0,2,0
2,0,20,0,85,0,1,10,1,0,3,...,1,0,0,1,0,0,0,0,2,0
3,1,25,2,54,0,1,5,0,1,0,...,0,0,0,0,0,0,0,0,2,0
4,1,21,0,73,0,1,15,0,0,3,...,1,0,1,0,0,0,1,0,0,0


In [10]:
#Slicing dataset
X = data.drop(['Selfharm'], axis = 1)
y = data['Selfharm']

In [11]:
# splitting X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

<h1>Gaussian Naive Bayes</h1>

In [13]:
#Gaussian
# training the model on training set 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
# making predictions on the testing set 
y_pred = gnb.predict(X_test) 
gnb_accy_train = round(accuracy_score(gnb.predict(X_train), y_train), 2)
gnb_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of Gaussian on training set: {:.2f}'.format(gnb_accy_train))
print ('Accuracy of Gaussian on test set: {:.2f}'.format(gnb_accy))

Accuracy of Gaussian on training set: 0.93
Accuracy of Gaussian on test set: 0.95


<h1>Stochastic Gradient Descent (SGD)</h1>

In [16]:
#Stochastic gradient descent
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier(loss='modified_huber', shuffle=True, random_state=101)
sgd.fit(X_train, y_train)
y_pred=sgd.predict(X_test)
sgd_accy_train = round(accuracy_score(sgd.predict(X_train), y_train), 2)
sgd_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of Stochastic gradient descent on training set: {:.2f}'.format(sgd_accy_train))
print ('Accuracy of Stochastic gradient descent on test set: {:.2f}'.format(sgd_accy))

Accuracy of Stochastic gradient descent on training set: 0.92
Accuracy of Stochastic gradient descent on test set: 0.95


<h1>Logistic Regression </h1>

In [17]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
lr_accy_train = round(accuracy_score(lr.predict(X_train), y_train), 2)
lr_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of Logistic Regression on training set: {:.2f}'.format(lr_accy_train))
print ('Accuracy of Logistic Regression on test set: {:.2f}'.format(lr_accy))

Accuracy of Logistic Regression on training set: 0.97
Accuracy of Logistic Regression on test set: 0.96


<h1>K Nearest Neighbour  (KNN)</h1>

In [18]:
#knn
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
knn_accy_train = round(accuracy_score(knn.predict(X_train), y_train), 2)
knn_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of KNN on training set: {:.2f}'.format(knn_accy_train))
print ('Accuracy of KNN on test set: {:.2f}'.format(knn_accy))

Accuracy of KNN on training set: 0.84
Accuracy of KNN on test set: 0.87


<h1>Decision Tree Classifier</h1>

In [19]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier(max_depth=10, random_state=101,
                            max_features=None, min_samples_leaf=15)
dtree.fit(X_train, y_train)
y_pred=dtree.predict(X_test)
dtree_accy_train = round(accuracy_score(dtree.predict(X_train), y_train), 2)
dtree_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of decision tree on training set: {:.2f}'.format(dtree_accy_train))
print ('Accuracy of decision tree on test set: {:.2f}'.format(dtree_accy))

Accuracy of decision tree on training set: 0.94
Accuracy of decision tree on test set: 0.95


<h1>Random Forest Classifier</h1>

In [21]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rfm=RandomForestClassifier(n_estimators=100, oob_score=True, n_jobs=-1,
                          random_state=101,max_features=25, min_samples_leaf=4)
rfm.fit(X_train, y_train)
y_pred=rfm.predict(X_test)
rfm_accy_train = round(accuracy_score(rfm.predict(X_train), y_train), 2)
rfm_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of Random Forest  on training set: {:.2f}'.format(rfm_accy_train))
print ('Accuracy of Random Forest  on test set: {:.2f}'.format(rfm_accy))

Accuracy of Random Forest  on training set: 0.98
Accuracy of Random Forest  on test set: 0.96


<h1>Support Vector Machine (SVM)</h1>

In [22]:
#svm
from sklearn.svm import SVC
svm=SVC(kernel='linear',C=0.025, random_state=101)
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
svm_accy_train = round(accuracy_score(svm.predict(X_train), y_train), 2)
svm_accy = round(accuracy_score(y_pred, y_test), 2)
print ('Accuracy of SVM on training set: {:.2f}'.format(svm_accy_train))
print ('Accuracy of SVM on test set: {:.2f}'.format(svm_accy))


Accuracy of SVM on training set: 0.93
Accuracy of SVM on test set: 0.96


<h1>Performance of Each Algorithm</h1>

Since Random Forest Classifier Algorithm gives better performance. We will be training our project based on it

In [24]:
performance = {'Gaussian Naive Bayes':[gnb_accy_train, gnb_accy],
              'Stochastic Gradient Descent(SGD)':[sgd_accy_train, sgd_accy],
              'Logistic Regression':[lr_accy_train, lr_accy],
              'K Nearest Neighbour (KNN)':[knn_accy_train, knn_accy],
              'Decision Tree Classifier':[dtree_accy_train, dtree_accy],
              'Random Forest Classifier':[rfm_accy_train,rfm_accy],
              'Support Vector Machine (SVM)' :[svm_accy_train,svm_accy]}

df_performance = pd.DataFrame.from_dict(performance, orient = 'index')
df_performance.columns=['Accuracy on Training Set','Accuracy on Test Set']

df_performance

,Accuracy on Training Set,Accuracy on Test Set
Gaussian Naive Bayes,0.93,0.95
Stochastic Gradient Descent(SGD),0.92,0.95
Logistic Regression,0.97,0.96
K Nearest Neighbour (KNN),0.84,0.87
Decision Tree Classifier,0.94,0.95
Random Forest Classifier,0.98,0.96
Support Vector Machine (SVM),0.93,0.96
